In [ ]:
from transformers import AutoTokenizer , AutoModelForCausalLM
import torch

In [ ]:
model = AutoModelForCausalLM.from_pretrained('Model')

tokenizer = AutoTokenizer.from_pretrained('Tokenizer')

In [ ]:
import speech_recognition as sr

def recognize_speech():
    # Initialize the recognizer
    recognizer = sr.Recognizer()

    # Use the microphone as an audio source
    with sr.Microphone() as source:
        print("Say something...")
        recognizer.adjust_for_ambient_noise(source)  # Adjust for ambient noise
        audio = recognizer.listen(source)

    try:
        # Recognize speech using a speech recognition API
        text = recognizer.recognize_google(audio)
        return text
    except sr.UnknownValueError:
        print("Could not understand audio")
        return None
    except sr.RequestError as e:
        print(f"Could not request results: {e}")
        return None

In [ ]:
def chat_bot():
    while True:
        user_input =  input("You: ")
        if user_input.lower() == 'exit':
            break
        elif user_input.lower() == 'micbutton':
            # Use speech recognition and pass the recognized text as user input
            recognized_text = recognize_speech()
            if recognized_text:
                user_input = recognized_text
            else:
                continue  # Skip the chat_bot loop if speech recognition fails
        user_input = "<USER>" + user_input + "<ASSISTANT>"
        temperature = 0.7
        max_response_length = 168
        input_ids = tokenizer.encode(user_input, truncation=True, return_tensors='pt')
        with torch.no_grad():
            response = model.generate(input_ids, pad_token_id=tokenizer.eos_token_id, max_length=max_response_length, temperature=temperature)
        bot_response = tokenizer.decode(response[0], skip_special_tokens=True)
        bot_response = bot_response[len(user_input):].strip()
        print("Bot: " + bot_response)

In [ ]:
chat_bot()